In [6]:
import folium
import panel as pn
import pandas as pd
import geopandas as gpd
import numpy as np

In [14]:
import pandas as pd
import geopandas as gpd
import folium
import panel as pn
import numpy as np
from folium import Map

In [10]:
cd ../maps

C:\Users\enisb\python\chatGPT\dash\maps


In [11]:
counties = gpd.read_file('US_County_Boundaries/US_CountyBndrys.shp') 
 
# counties[counties['NAME'].isin(["Erie", 'Niagara'])]
county = counties[counties['GEOID'].isin(['36029', '36063'])] #"Erie", 'Niagara'
county_wm = county.to_crs(epsg=3857)


In [16]:
# Add custom script to disable scroll wheel zoom
disable_scroll_js = """
    <script>
        var map = document._leaflet_map_;
        map.scrollWheelZoom.disable();
    </script>
"""

In [17]:
# Sample data
data_dict = {
    'name': ['BOA 1', 'BOA 2', 'BOA 3', 'BOA 4', 'BOA 5', 'KB 1', 'KB 2', 'KB 3', 'KB 4', 'KB 5'],
    'lon': [-78.83101, -78.85, -78.80, -78.82, -78.83, -78.88, -78.90, -78.92, -78.95, -78.96],
    'lat': [42.71618, 42.70, 42.72, 42.74, 42.75, 42.77, 42.78, 42.79, 42.76, 42.73],
    'bank': ['Bank of America', 'Bank of America', 'Bank of America', 'Bank of America', 'Bank of America',
             'KeyBank', 'KeyBank', 'KeyBank', 'KeyBank', 'KeyBank']
}

data = pd.DataFrame(data_dict)

# Convert to GeoDataFrame
addresses = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.lon, data.lat))
addresses.crs = "EPSG:4326"
addresses_wm = addresses.to_crs(epsg=3857)  # Convert addresses to the same CRS

buffered_addresses = addresses_wm.copy()
buffered_addresses['geometry'] = addresses_wm.buffer(1609.34 *1)
buffered_addresses

# Transform buffered addresses back to EPSG:4326
buffered_addresses_4326 = buffered_addresses.to_crs(epsg=4326)
# ... Your data setup code here ...

# Adding a random 'Open on Saturday' column for demonstration purposes
np.random.seed(0)
addresses['open_on_saturday'] = np.random.choice([True, False], addresses.shape[0])


addresses_wm = addresses.to_crs(epsg=3857)  # Convert addresses to the same CRS

buffered_addresses = addresses_wm.copy()
buffered_addresses['geometry'] = addresses_wm.buffer(1609.34 *1)
buffered_addresses

# Transform buffered addresses back to EPSG:4326
buffered_addresses_4326 = buffered_addresses.to_crs(epsg=4326)

In [45]:
 

def generate_map(bank='All', saturday=False, radius=1):
    m = Map(location=[addresses['lat'].mean(), addresses['lon'].mean()], zoom_start=10)
    
    # Add county layer
    folium.GeoJson(county_wm).add_to(m)
    
    # Filter based on selected bank
    if bank != 'All':
        filtered_data = buffered_addresses_4326[addresses['bank'] == bank]
    else:
        filtered_data = buffered_addresses_4326
        
    # Filter based on Saturday open
    if saturday:
        filtered_data = filtered_data[addresses['open_on_saturday']]
        
    # Colors for banks
    bank_colors = {
        'Bank of America': 'blue',
        'KeyBank': 'red'
    }
    
    # Add circles for each buffered address
    for _, row in filtered_data.iterrows():
        folium.Circle(
            location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
            radius=1609.34 * radius,
            color=None,
            fill=True,
            fill_color=bank_colors[row['bank']],
            popup=f"Bank: {row['bank']}<br>Open on Saturday: {row['open_on_saturday']}"
        ).add_to(m)
    
    m.get_root().html.add_child(folium.Element(disable_scroll_js))

    return m

 
def folium_to_html(map_obj):
    """Converts a Folium Map object to an HTML string."""
    html = map_obj._repr_html_()
    return html

bank_selector = pn.widgets.RadioButtonGroup(name='Bank', options=addresses['bank'].unique().tolist() + ['All'], value='Bank of America')
saturday_selector = pn.widgets.Checkbox(name='Open on Saturday')

initial_map = generate_map(bank_selector.value, saturday_selector.value)
map_html = folium_to_html(initial_map)
map_pane = pn.pane.HTML(map_html, width=1900, height=1080)

def update(event):
    updated_map = generate_map(bank_selector.value, saturday_selector.value, radius_selector.value)
    map_pane.object = folium_to_html(updated_map)

bank_selector.param.watch(update, 'value')
saturday_selector.param.watch(update, 'value')
radius_selector.param.watch(update, 'value')


sidebar = pn.Column(bank_selector, saturday_selector, radius_selector, text)
template = pn.template.BootstrapTemplate(
    title='Branch Dashboard',
    sidebar=sidebar,
) 

maincontent = pn.Column(map_pane, margin=0 )
template.main.append(maincontent)

template.show()

Launching server at http://localhost:63082


C:\Users\enisb\anaconda3\envs\panel\lib\site-packages\geopandas\geodataframe.py:1475: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\enisb\anaconda3\envs\panel\lib\site-packages\geopandas\geodataframe.py:1475: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [43]:
pn.extension()
radius_selector = pn.widgets.Select(name='Sort by:', options=list(range(1,11)), value=1, width=60)

In [31]:
pn.widgets.Select?

Init signature: pn.widgets.Select(**params)
Docstring:     
params(description=String, disabled_options=List, groups=Dict, size=Integer, value=Parameter, options=ClassSelector, disabled=Boolean, loading=Boolean, align=Align, aspect_ratio=Aspect, background=Parameter, css_classes=List, design=ObjectSelector, height=Integer, height_policy=ObjectSelector, margin=Margin, max_height=Integer, max_width=Integer, min_height=Integer, min_width=Integer, sizing_mode=ObjectSelector, styles=Dict, stylesheets=List, tags=List, visible=Boolean, width=Integer, width_policy=ObjectSelector, name=String)

    The `Select` widget allows selecting a value from a list or dictionary of
    `options` by selecting it from a dropdown menu or selection area.

    It falls into the broad category of single-value, option-selection widgets
    that provide a compatible API and include the `RadioBoxGroup`,
    `AutocompleteInput` and `DiscreteSlider` widgets.

    Reference: https://panel.holoviz.org/reference/widget

In [22]:
text = pn.pane.Markdown("Select which bank branches to view. Also option is to select the branches open on Saturday only.")

In [27]:
pn.Column?

Init signature: pn.Column(*objects: 'Any', **params: 'Any')
Docstring:     
params(scroll=Boolean, objects=List, loading=Boolean, align=Align, aspect_ratio=Aspect, background=Parameter, css_classes=List, design=ObjectSelector, height=Integer, height_policy=ObjectSelector, margin=Margin, max_height=Integer, max_width=Integer, min_height=Integer, min_width=Integer, sizing_mode=ObjectSelector, styles=Dict, stylesheets=List, tags=List, visible=Boolean, width=Integer, width_policy=ObjectSelector, name=String)

    The `Column` layout allows arranging multiple panel objects in a vertical
    container.

    It has a list-like API with methods to `append`, `extend`, `clear`,
    `insert`, `pop`, `remove` and `__setitem__`, which makes it possible to
    interactively update and modify the layout.

    Reference: https://panel.holoviz.org/reference/layouts/Column.html

    :Example:

    >>> pn.Column(some_widget, some_pane, some_python_object)
    
Parameters of 'Column'

Parameters changed f